In [52]:
import pandas as pd
from kagglehub import load_dataset, KaggleDatasetAdapter
import numpy as np

In [29]:
import pyarrow.feather as feather

table = feather.read_table("dataset/featured_data_with_clusters.feather")
data = table.to_pandas()
data.head()

,title,track_id,song_embedding,cluster
0,Food,2,"[1.0710538215395269, 0.10972996075557472, -0.2...",11
1,Electric Ave,3,"[-0.14778286108405392, -0.17472288765382982, -...",3
2,This World,5,"[-0.09108703035051309, -0.25107759845873573, -...",3
3,Freeway,10,"[0.08307237013799296, -0.140689227131485, 0.06...",3
4,Spiritual Level,20,"[-0.14975542477189308, -0.26310469120550006, -...",9


# ***Genre Extraction of all songs***

In [6]:
file_path = "fma_metadata/tracks.csv"
song_tracks = load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "imsparsh/fma-free-music-archive-small-medium",
  file_path,
)

C:\Users\SOUMYOJEET\AppData\Local\Temp\ipykernel_8916\2051274980.py:2: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  song_tracks = load_dataset(
c:\Users\SOUMYOJEET\miniconda3\Lib\site-packages\kagglehub\pandas_datasets.py:91: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [16]:
song_tracks.head()

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,...,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World


In [ ]:
song_tracks[[col for col in song_tracks.columns if 'track' in col]].head()


,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World


In [40]:
# cleaning the raw track data
col_names = ['track_id', song_tracks.loc[0, 'track.7']]
data_ = song_tracks.loc[2:, ['Unnamed: 0', 'track.7']]
data_.columns = col_names
data_['track_id'] = data_['track_id'].astype('int32')
data_.reset_index(drop=True, inplace=True)
data_.head()


,track_id,genre_top
0,2,Hip-Hop
1,3,Hip-Hop
2,5,Hip-Hop
3,10,Pop
4,20,NaN


In [46]:
data_.loc[data_['track_id'] == 2, 'genre_top'].values[0]

'Hip-Hop'

In [ ]:
def featured_data_with_genres(featured_data):
    genres = []
    for series in featured_data.itertuples():
        genre = data_.loc[data_['track_id'] == series.track_id, 'genre_top'].values[0]
        genres.append(genre)
    
    featured_data['genre_top'] = genres
    return featured_data
    

In [57]:
featured_data_with_genres(data).isnull().sum()

title               0
track_id            0
song_embedding      0
cluster             0
genre_top         160
dtype: int64

In [56]:
data.head()

,title,track_id,song_embedding,cluster,genre_top
0,Food,2,"[1.0710538215395269, 0.10972996075557472, -0.2...",11,Hip-Hop
1,Electric Ave,3,"[-0.14778286108405392, -0.17472288765382982, -...",3,Hip-Hop
2,This World,5,"[-0.09108703035051309, -0.25107759845873573, -...",3,Hip-Hop
3,Freeway,10,"[0.08307237013799296, -0.140689227131485, 0.06...",3,Pop
4,Spiritual Level,20,"[-0.14975542477189308, -0.26310469120550006, -...",9,NaN


# ***Final Recommendation Pipeline***

In [72]:
import joblib
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
# Spliting dataset into train and test
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data.shape, test_data.shape

((1574, 5), (175, 5))

In [76]:
train_data.reset_index(drop=True, inplace=True)
train_data.head()

,title,track_id,song_embedding,cluster,genre_top
0,Haul,3258,"[-0.1926710115387628, -0.28993842167841277, -0...",4,NaN
1,Hops,1754,"[-0.1443107336701126, -0.3201891391901447, -0....",2,Rock
2,Track 13,168,"[-0.2895501532949549, -0.154811141925688, -0.2...",0,Experimental
3,I Can't Tell In His Eyes,4536,"[-0.15885646261696243, -0.2159091820737676, -0...",2,Blues
4,My Name Is Robert,3557,"[-0.08759853059011254, -0.16779158143712342, 0...",2,Electronic


In [68]:
test_data.reset_index(drop=True, inplace=True)
test_data.head()

,title,track_id,song_embedding,cluster,genre_top
0,Morbidity,1309,"[0.3111036350391608, 2.603706017700491, 5.5608...",13,Electronic
1,My Baby,3811,"[-0.13846137113569337, -0.12423147883974683, -...",3,Rock
2,In Tongues,1711,"[0.4507531350269054, 0.24565221782419627, 0.21...",7,Rock
3,Ding dang (skat),345,"[-0.19676282758297636, -0.2402360322304917, -0...",6,Rock
4,Lovin' Machine,744,"[-0.23195902148735878, -0.27584624682483905, -...",2,Soul-RnB


In [60]:
test_data.isnull().sum()

title              0
track_id           0
song_embedding     0
cluster            0
genre_top         12
dtype: int64

In [88]:
def recc_top_k (train_data, target_song, k):
    # get all song embeddings in 2D array
    all_song_embeddings = np.vstack(train_data.loc[:, 'song_embedding'].values)

    # compute cosine similarity scores
    scores = cosine_similarity(all_song_embeddings, [target_song]).flatten()
    # getting top k indices
    idx = np.argsort(scores)[-k:]
    
    # getting songs corresponding to the indices
    return train_data.iloc[idx]
    

def algorithm(train_data, target_song_embedding, k):
    # loading model
    km_model = joblib.load('models/kmeans_model.pkl')

    # predicting cluster for the target song
    cluster_label = km_model.predict([target_song_embedding])[0]

    # getting songs from the same cluster
    cluster_songs = train_data[train_data['cluster'] == cluster_label]
    return recc_top_k(cluster_songs, target_song_embedding, k)
    

In [89]:
algorithm(train_data, test_data.loc[0, 'song_embedding'], 5)

,title,track_id,song_embedding,cluster,genre_top
1217,Rabidity,1305,"[0.5220437171048603, 1.1662412443311891, 0.782...",13,Electronic
510,Flacidity,1324,"[0.8764188671095794, 1.2071538638572026, 1.476...",13,Electronic
474,Rigidity,1335,"[0.06620460021591998, 4.024146743976975, 2.478...",13,Electronic
445,Death,3286,"[0.6729285682304638, 1.46863178616545, 1.65170...",13,NaN
527,Acidity,1315,"[0.27406221943936476, 1.5572512644108059, 0.87...",13,Electronic


In [92]:
test_data.loc[0, 'genre_top']

'Electronic'

In [93]:
recc_top_k(train_data, test_data.loc[0, 'song_embedding'], 5)

,title,track_id,song_embedding,cluster,genre_top
474,Rigidity,1335,"[0.06620460021591998, 4.024146743976975, 2.478...",13,Electronic
445,Death,3286,"[0.6729285682304638, 1.46863178616545, 1.65170...",13,NaN
494,Childs Eye,1279,"[0.13529193886170787, 0.4978030766583059, 0.23...",1,Rock
527,Acidity,1315,"[0.27406221943936476, 1.5572512644108059, 0.87...",13,Electronic
281,Rancidity,1327,"[0.6296269942211751, 0.9575747376971948, 0.667...",1,Electronic


- KMeans + Cosine Similarity is almost 5x faster than Cosine Similarity alone